# Ola Bike Ride Request Demand Forecast

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime
from datetime import date
import calendar

In [7]:
# from google.colab import files
# uploaded = files.upload()
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import io
import os

# Create a file upload widget
uploader = widgets.FileUpload(
    accept='.csv,.xlsx,.xls,.json',  # Accept CSV, Excel, and JSON files
    multiple=False  # Set to True if you need multiple files
)

# Display the widget
display(uploader)

# Function to process uploaded file
def handle_upload(change):
    for filename, file_info in uploader.value.items():
        print(f"Uploaded file: {filename}")
        try:
            content = file_info['content']
            # Determine file type based on extension
            file_extension = os.path.splitext(filename)[1].lower()
            
            if file_extension == '.csv':
                df = pd.read_csv(io.BytesIO(content))
            elif file_extension in ['.xlsx', '.xls']:
                df = pd.read_excel(io.BytesIO(content))
            elif file_extension == '.json':
                df = pd.read_json(io.BytesIO(content))
            else:
                print(f"Unsupported file type: {file_extension}")
                return
            
            print("Dataset Preview:")
            print(df.head())
            # Save the file locally (optional)
            with open(filename, 'wb') as f:
                f.write(content)
            # Assign to global variable for further use
            global dataset
            dataset = df
            # Example: Initial preprocessing steps
            print("\nNull Values:")
            print(dataset.isnull().sum())
            # Example: Convert datetime column (adjust column name as needed)
            if 'datetime' in dataset.columns:
                dataset['datetime'] = pd.to_datetime(dataset['datetime'])
                print("\nDatetime column converted to:", dataset['datetime'].dtype)
        except Exception as e:
            print(f"Error loading file: {e}")

# Bind the function to the uploader
uploader.observe(handle_upload, names='value')



FileUpload(value={}, accept='.csv,.xlsx,.xls,.json', description='Upload')

SyntaxError: invalid syntax (4053252937.py, line 1)

In [ ]:
train_df = pd.read_csv('train.csv')

In [ ]:
# train_df

In [ ]:
train_df.shape

In [ ]:
train_df.info()

### Dealing with Null Values

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.columns

In [ ]:
train_df.dtypes

### Changing into datetime format

In [ ]:
# Define your function to convert a column to datetime
def change_into_datetime(col):
    train_df[col] = pd.to_datetime(train_df[col], format="%d-%m-%Y %H:%M")

# Assuming `train_df` is your DataFrame
# Call the function for the 'datetime' column
change_into_datetime('datetime')

# Check if the conversion is successful
print(train_df['datetime'].head())


In [ ]:
train_df['year'] = train_df['datetime'].dt.year
train_df['month'] = train_df['datetime'].dt.month
train_df['date'] = train_df['datetime'].dt.date
train_df['hour'] = train_df['datetime'].dt.hour
train_df['day of the week'] = train_df['datetime'].dt.dayofweek

In [ ]:
train_df['year'].unique()

### Mapping key values to Dictionary

In [ ]:
dict = {2011 :'1', 2012 :'2'}

In [ ]:
train_df['year'] = train_df['year'].map(dict)
train_df['year'].unique()

In [ ]:
train_df.drop('datetime',axis=1,inplace=True)

In [ ]:
train_df.head()

## Performing Exploratory Data Analysis

In [ ]:
train_df.drop('count',axis=1).describe()

In [ ]:
sns.pairplot(train_df);

### Distribution of target variable 'count'

In [ ]:
sns.distplot(train_df['count']);

Inference - The target variable is right skewed and hence we have to take the log of the variable to check if the distribution becomes normal or not.

In [ ]:
sns.distplot(np.log(train_df['count']));

### Visualizing the Demand using multiple variables

###  Demand per day of the week

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=train_df, x = 'day of the week', y = 'count', palette='rainbow');

Inference -  demand of bike rentals were almost same for each day of the week. So this feature will not be useful in predicting the demand therefore we will have to drop this feature.

### Demand per month

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=train_df, x='month', y = 'count', palette = 'rainbow');

Inference - Demand for bike rentals was high during the months of summer and the demand drops during the months of winter.

### Year Month Demand

In [ ]:
group_year_month = train_df.groupby(['year','month'])
average_year_month = group_year_month['count'].mean()

In [ ]:
for year in average_year_month.index.levels[0]:
    plt.plot(average_year_month[year].index,average_year_month[year],label=year)

plt.legend()
plt.xlabel('Month')
plt.ylabel('Count')
plt.grid(True)
plt.title('Average Monthly Rental Count for 2011,2012')
plt.show()

### Demand per hour

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(data = train_df, x = 'hour', y = 'count', palette = 'rainbow');

Inference - Demand for bike rentals were high during the office hours and it was low during the non-working hours as it might be possible that most of the people use the bike rental services to reach their office premises or leave their premises during these hours.

### Year Hour Demand

In [ ]:
group_year_hour = train_df.groupby(['year','hour'])
average_year_hour = group_year_hour['count'].mean()

In [ ]:
for year in average_year_hour.index.levels[0]:
    plt.plot(average_year_hour[year].index,average_year_hour[year],label=year)

plt.legend()
plt.xlabel('Month')
plt.ylabel('Count')
plt.grid(True)
plt.title('Average Hourly Rental Count for 2011,2012')
plt.show()

### Average Hourly Count

In [ ]:
group_hour = train_df.groupby(['hour'])
average_by_hour = group_hour['count'].mean()

In [ ]:
plt.plot(average_by_hour.index,average_by_hour)
plt.xlabel('Date')
plt.ylabel('Rental Count')
plt.xticks(np.arange(24))
plt.grid(True)
plt.title('Average Hourly Rental Count')
plt.show()

### Demand per season

In [ ]:
plt.figure(figsize = (8,4))
sns.barplot(data = train_df, x = 'season', y = 'count', palette = 'rainbow');

Inference - Demand was high during the summer and fall seasons while it drops during winter and spring season as the weather during these seasons might not be suitable for bike rentals.

### Demand as per holidays

In [ ]:
plt.figure(figsize=(6,6))
sns.barplot(data = train_df, x = 'holiday', y = 'count', palette = 'rainbow');

Inference - Demand was high during working days and low during holidays.

### Demand on working days

In [ ]:
plt.figure(figsize=(6,4))
sns.barplot(data = train_df, x = 'workingday', y = 'count');

Inference - It doesn't affect the demand much as it remains almost the same both the time, therefore we will have to drop this column.

### Working day Hour Demand

In [ ]:
group_workingday_hour = train_df.groupby(['workingday','hour'])
average_workingday_hour = group_workingday_hour['count'].mean()

In [ ]:
for workingday in average_workingday_hour.index.levels[0]:
    plt.plot(average_workingday_hour[workingday].index,average_workingday_hour[workingday],label=workingday)

plt.legend()
plt.xlabel('Hour')
plt.ylabel('Count')
plt.xticks(np.arange(24))
plt.grid(True)
plt.title('Average Hourly Rental Count for by Working Day')
plt.show()

### Visualization of the continuous varibles using Histogram

###### Continuous features - temp, atemp, humidity, windspeed, count

In [ ]:
train_df.temp.unique()
fig,axes=plt.subplots(2,2)
axes[0,0].hist(x="temp",data=train_df,edgecolor="black",linewidth=2,color='#a0e0b6')
axes[0,0].set_title("Variation of temp")
axes[0,1].hist(x="atemp",data=train_df,edgecolor="black",linewidth=2,color='#85b98d')
axes[0,1].set_title("Variation of atemp")
axes[1,0].hist(x="windspeed",data=train_df,edgecolor="black",linewidth=2,color='#3e936e')
axes[1,0].set_title("Variation of windspeed")
axes[1,1].hist(x="humidity",data=train_df,edgecolor="black",linewidth=2,color='#335d40')
axes[1,1].set_title("Variation of humidity")
fig.set_size_inches(10,10)

### Temperature vs Demand

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data = train_df, x = 'temp', y = 'count');

##### Since this is hard to visualze. a better way is to convert the 'temp' variable into intervals or so called bins and then treat it like a discrete variable.

In [ ]:

# Assuming train_df is your DataFrame
viz_df = train_df.copy()

# Describe the 'temp' column
viz_df['temp'].describe()

# Create a new column 'temp_bin' with discrete values of 'temp'
viz_df['temp_bin'] = np.floor(viz_df['temp'] / 5).astype(int)

# Check the unique values in 'temp_bin'
print(viz_df['temp_bin'].unique())

# Visualize 'temp_bin' against 'count' using seaborn catplot
sns.catplot(x="temp_bin", y="count", data=viz_df, kind='bar', height=6, aspect=1.5)
plt.xlabel('Temperature Bin')
plt.ylabel('Count')
plt.title('Count vs. Temperature Bin')
plt.show()

Inference - The demand is highest for bins 6 and 7 which is about tempearure  30-35(bin 6) and 35-40 (bin 7).

### Demand vs aTemp

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data = train_df, x = 'atemp', y = 'count');

In [ ]:
# Assuming train_df is your DataFrame
viz_df = train_df.copy()

# Describe the 'atemp' column
print(viz_df['atemp'].describe())

# Create a new column 'temp_bin' with discrete values of 'atemp'
viz_df['temp_bin'] = np.floor(viz_df['atemp'] / 5).astype(int)

# Check the unique values in 'temp_bin'
print(viz_df['temp_bin'].unique())

# Visualize 'temp_bin' against 'count' using seaborn catplot
sns.catplot(x="temp_bin", y="count", data=viz_df, kind='bar', height=6, aspect=1.5)
plt.xlabel('Temperature Bin')
plt.ylabel('Count')
plt.title('Count vs. Temperature Bin')
plt.show()

##### Inference  - We can see that this plot is almost similar to the previous plot of temperature and demand which means there is a high correlation between temp and atemp features.

### Humidity vs Demand

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data = train_df, x = 'humidity', y='count');

In [ ]:
# Assuming train_df is your DataFrame
viz_df = train_df.copy()

# Describe the 'humidity' column
print(viz_df['humidity'].describe())

# Create a new column 'temp_bin' with discrete values of 'humidity'
viz_df['temp_bin'] = np.floor(viz_df['humidity'] / 10).astype(int)

# Check the unique values in 'temp_bin'
print(viz_df['temp_bin'].unique())

# Visualize 'temp_bin' against 'count' using seaborn catplot
sns.catplot(x="temp_bin", y="count", data=viz_df, kind='bar', height=6, aspect=1.5)
plt.xlabel('Humidity Bin')
plt.ylabel('Count')
plt.title('Count vs. Humidity Bin')
plt.show()

Inference - Humidity is not much correlated to Demand

### Windspeed vs Demand

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data = train_df, x = 'windspeed', y='count');

In [ ]:
# Assuming train_df is your DataFrame
viz_df = train_df.copy()

# Describe the 'windspeed' column
print(viz_df['windspeed'].describe())

# Create a new column 'temp_bin' with discrete values of 'windspeed'
viz_df['temp_bin'] = np.floor(viz_df['windspeed'] / 5).astype(int)

# Check the unique values in 'temp_bin'
print(viz_df['temp_bin'].unique())

# Visualize 'temp_bin' against 'count' using seaborn catplot
sns.catplot(x="temp_bin", y="count", data=viz_df, kind='bar', height=6, aspect=1.5)
plt.xlabel('Windspeed Bin')
plt.ylabel('Count')
plt.title('Count vs. Windspeed Bin')
plt.show()

Inference - Since windspeed has a very low correlation with the demand so it does not affect the demand much.

### Visualization of the Correlation Matrix of continuous features and understanding how it will affect the Target variable

In [ ]:
corrdata = train_df[["temp","atemp","humidity","windspeed","count"]]
corrmat = corrdata.corr()
plt.figure(figsize=(10,8))
sns.heatmap(corrmat, annot = True, cmap= 'winter');

### Visualization of the Correlation Matrix of all the independent features

In [ ]:

# Filter out non-numeric columns
numeric_df = train_df.select_dtypes(include=[np.number])

# Calculate the correlation matrix
cor_mat = numeric_df.corr()

# Create a mask for the upper triangle
mask = np.triu(np.ones_like(cor_mat, dtype=bool))

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(30, 12))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(cor_mat, mask=mask, square=True, annot=True, cbar=True, ax=ax)

# Display the heatmap
plt.show()

## Data Preprocessing

In [ ]:
train_df

In [ ]:
train_df["year"] = pd.to_numeric(train_df["year"])

### Performing One Hot Encoding on categorical features

In [ ]:
season = pd.get_dummies(train_df['season'],prefix='season',drop_first=True)
weather = pd.get_dummies(train_df['weather'],prefix='weather',drop_first=True)
holiday = pd.get_dummies(train_df['holiday'],prefix='holiday',drop_first=True)
month = pd.get_dummies(train_df['month'],prefix='month',drop_first=True)
hour = pd.get_dummies(train_df['hour'],prefix='hour',drop_first=True)
train_df = pd.concat([train_df,season,weather,holiday,month,hour],axis=1)
train_df.drop(['season','weather','holiday','month','hour'], axis=1,inplace=True)

In [ ]:
pd.set_option('display.max_columns',52)
train_df.head()

### Dropping the features with low correlation

In [ ]:
train_df.drop(['date','day of the week','year','windspeed','workingday'], axis=1, inplace=True)

In [ ]:
train_df.head()

In [ ]:
train_df.columns.to_series().groupby(train_df.dtypes).groups

In [ ]:
train_df.dtypes

### Visualization of the Correlation Matrix of the preprocessed data

In [ ]:
cor_mat= train_df[:].corr()
mask = np.array(cor_mat)
mask[np.tril_indices_from(mask)] = False
fig=plt.gcf()
fig.set_size_inches(50,50)
sns.heatmap(data=cor_mat,mask=mask,square=True,annot=True,cbar=True)

### Splitting the data

In [ ]:
X = train_df.drop('count',axis=1)

In [ ]:
X.shape

In [ ]:
y = np.log(train_df['count'])

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2)

### Using Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
linear_rg = LinearRegression()

In [ ]:
linear_rg.fit(X_train,y_train)

In [ ]:
preds1 = linear_rg.predict(X_test)

In [ ]:
preds1

In [ ]:
from sklearn import metrics

In [ ]:
r2_score = metrics.r2_score(y_test,preds1)
print('r2_score is:{}'.format(r2_score))

print('MAE:',metrics.mean_absolute_error(y_test,preds1))
print('MSE:',metrics.mean_squared_error(y_test,preds1))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test,preds1)))
sns.distplot(y_test-preds1)

#### Accuracy - 86.55%

### Using Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dt_rg = DecisionTreeRegressor(max_depth=5)

In [ ]:
dt_rg.fit(X_train,y_train)

In [ ]:
preds2 = dt_rg.predict(X_test)

In [ ]:
preds2

In [ ]:
r2_score = metrics.r2_score(y_test,preds2)
print('r2_score is:{}'.format(r2_score))

print('MAE:',metrics.mean_absolute_error(y_test,preds2))
print('MSE:',metrics.mean_squared_error(y_test,preds2))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test,preds2)))
sns.distplot(y_test-preds2)

#### Accuracy -  99.03%

### Using Hypertuned KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
n_neighbors=[]
for i in range (0,50,5):
    if(i!=0):
        n_neighbors.append(i)
params_dict={'n_neighbors':n_neighbors,'n_jobs':[-1]}
clf_knn = GridSearchCV(estimator=KNeighborsRegressor(),param_grid=params_dict,scoring='neg_mean_squared_log_error')
clf_knn.fit(X_train,y_train)
preds3 = clf_knn.predict(X_test)

In [ ]:
preds3

In [ ]:
r2_score = metrics.r2_score(y_test,preds3)
print('r2_score is:{}'.format(r2_score))

print('MAE:',metrics.mean_absolute_error(y_test,preds3))
print('MSE:',metrics.mean_squared_error(y_test,preds3))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test,preds3)))
sns.distplot(y_test-preds3)

#### Accuracy - 99.32%

### Using Hypertuned Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
no_of_test=[500]
params_dict={'n_estimators':no_of_test,'n_jobs':[-1],'max_features':["auto",'sqrt','log2']}
clf_rf = GridSearchCV(estimator=RandomForestRegressor(),param_grid=params_dict,scoring='neg_mean_squared_log_error')
clf_rf.fit(X_train,y_train)
preds4 = clf_rf.predict(X_test)

In [ ]:
preds4

In [ ]:
r2_score = metrics.r2_score(y_test,preds4)
print('r2_score is:{}'.format(r2_score))

print('MAE:',metrics.mean_absolute_error(y_test,preds4))
print('MSE:',metrics.mean_squared_error(y_test,preds4))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test,preds4)))
sns.distplot(y_test-preds4)

#### Accuracy - 99.995%

### Using Hypertuned XGBoost

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }
    XGB = XGBRegressor()

    gridsearch = GridSearchCV(estimator = XGB,
                           param_grid = param_tuning,
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gridsearch.fit(X_train,y_train)

    return gridsearch.best_params

In [ ]:
XGB = XGBRegressor(
        objective = 'reg:squarederror',
        colsample_bytree = 0.5,
        learning_rate = 0.05,
        max_depth = 6,
        min_child_weight = 1,
        n_estimators = 1000,
        subsample = 0.7)

XGB.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test),(X_test,y_test)], verbose=False)

In [ ]:
preds5 = XGB.predict(X_test)

In [ ]:
preds5

In [ ]:
r2_score = metrics.r2_score(y_test,preds5)
print('r2_score is:{}'.format(r2_score))

print('MAE:',metrics.mean_absolute_error(y_test,preds5))
print('MSE:',metrics.mean_squared_error(y_test,preds5))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test,preds5)))
sns.distplot(y_test-preds3)

#### Accuracy - 99.97%

## After performing the model comparative analysis we can conclude that almost all the models had an accuracy above 99% except the Linear Regressor. The Hypertuned Random Forest Regressor gave best result with 99.995% accuracy

### Saving the Hypertuned Random Forest Regressor and XGBoost Regressor models to get predictions from the test set.